# Event chain aggregation
Program for aggregating outcomes to form event chains based on simulated battle data.

#### Reading and cleaning the data
We start of with importing the csv files containing the simulation results. Then we concatenate the files to a singular dataframe, create success rate variable and rename some of the columns.

In [27]:
import pandas as pd
import ast
import networkx as nx

# Load simulation results and process DataFrame.
df1 = pd.read_csv('M1_simulation_results.csv')
df2 = pd.read_csv('M2_simulation_results.csv')
df3 = pd.read_csv('M3_simulation_results.csv')

# Combine and preprocess simulation data.
df = pd.concat(objs=[df1, df2, df3], join='inner')
df.columns = ['matix', 'initial_state', 'final_state', 'new_state', 'successful']
df = df.value_counts().reset_index(name='count')
df['fraction_of_init'] = df['count'] / 1000

df_dicts = df.copy()
for col in ['initial_state', 'final_state', 'new_state']:
    df_dicts[col] = df_dicts[col].map(ast.literal_eval)
    df[f'{col}_dict'] = df_dicts[col]

df.rename(columns={'initial_state': 'initial_state_str', 'final_state': 'final_state_str',
    'new_state': 'new_state_str'}, inplace=True)
df['aggr_prob'] = 0
df.sort_values(by='initial_state_str')
df

,matix,initial_state_str,final_state_str,new_state_str,successful,count,fraction_of_init,initial_state_dict,final_state_dict,new_state_dict,aggr_prob
0,M1,"{0: [2, 3], 1: [1, 1]}","{0: [2, 3], 1: [1, 1]}","{0: [2, 3], 1: [1, 1]}",0,1000,1.000,"{0: [2, 3], 1: [1, 1]}","{0: [2, 3], 1: [1, 1]}","{0: [2, 3], 1: [1, 1]}",0
1,M3,"{0: [2, 4], 1: [1, 1], 2: [1, 1], 3: [1, 1]}","{0: [2, 4], 1: [1, 1], 2: [1, 1], 3: [1, 1]}","{0: [2, 4], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0,1000,1.000,"{0: [2, 4], 1: [1, 1], 2: [1, 1], 3: [1, 1]}","{0: [2, 4], 1: [1, 1], 2: [1, 1], 3: [1, 1]}","{0: [2, 4], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0
2,M1,"{0: [2, 4], 1: [1, 1]}","{0: [2, 4], 1: [1, 1]}","{0: [2, 4], 1: [1, 1]}",0,1000,1.000,"{0: [2, 4], 1: [1, 1]}","{0: [2, 4], 1: [1, 1]}","{0: [2, 4], 1: [1, 1]}",0
3,M1,"{0: [2, 1], 1: [1, 1]}","{0: [2, 1], 1: [1, 1]}","{0: [2, 1], 1: [1, 1]}",0,1000,1.000,"{0: [2, 1], 1: [1, 1]}","{0: [2, 1], 1: [1, 1]}","{0: [2, 1], 1: [1, 1]}",0
4,M1,"{0: [2, 2], 1: [1, 1]}","{0: [2, 2], 1: [1, 1]}","{0: [2, 2], 1: [1, 1]}",0,1000,1.000,"{0: [2, 2], 1: [1, 1]}","{0: [2, 2], 1: [1, 1]}","{0: [2, 2], 1: [1, 1]}",0
...,...,...,...,...,...,...,...,...,...,...,...
1937,M3,"{0: [2, 1], 1: [1, 3], 2: [1, 4], 3: [1, 4]}","{0: [1, 0], 1: [1, 2], 2: [1, 1], 3: [1, 1]}","{0: [1, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",1,2,0.002,"{0: [2, 1], 1: [1, 3], 2: [1, 4], 3: [1, 4]}","{0: [1, 0], 1: [1, 2], 2: [1, 1], 3: [1, 1]}","{0: [1, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0
1938,M3,"{0: [2, 1], 1: [1, 3], 2: [1, 3], 3: [1, 4]}","{0: [1, 0], 1: [1, 1], 2: [1, 2], 3: [1, 1]}","{0: [1, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",1,2,0.002,"{0: [2, 1], 1: [1, 3], 2: [1, 3], 3: [1, 4]}","{0: [1, 0], 1: [1, 1], 2: [1, 2], 3: [1, 1]}","{0: [1, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0
1939,M3,"{0: [2, 1], 1: [1, 4], 2: [1, 4], 3: [1, 3]}","{0: [2, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}","{0: [2, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0,1,0.001,"{0: [2, 1], 1: [1, 4], 2: [1, 4], 3: [1, 3]}","{0: [2, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}","{0: [2, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0
1940,M3,"{0: [2, 1], 1: [1, 3], 2: [1, 4], 3: [1, 3]}","{0: [1, 0], 1: [1, 1], 2: [1, 1], 3: [1, 2]}","{0: [1, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",1,1,0.001,"{0: [2, 1], 1: [1, 3], 2: [1, 4], 3: [1, 3]}","{0: [1, 0], 1: [1, 1], 2: [1, 1], 3: [1, 2]}","{0: [1, 1], 1: [1, 1], 2: [1, 1], 3: [1, 1]}",0


#### Creating the initial game setup
The example graph and information pertaining to node ownership and troop numbers is added as a dict. It will keep track of the game status for each event chain.

In [28]:
game_status_dict = {'original_graph': [
        [1, 1, 0, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 0, 0],
        [1, 1, 0, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1],],
    
    'node_ownership': {
        0: 'Player1',
        1: 'Player1',
        2: 'Player1',
        3: 'Player2',
        4: 'Player2',
        5: 'Player2',
        6: 'Player2',
        7: 'Player2'},
    
    'node_troops': {
        0: 4,
        1: 4,
        2: 4,
        3: 2,
        4: 2,
        5: 2,
        6: 2,
        7: 2},
    'aggr_node_path': [], 'aggr_prob': 0, 'aggr_attack_type' : [], 'aggr_attacking_nodes': [], 'event_probs': []}

#game_status_dict

#### Identify enemy nodes
The function "find_adjacent_enemy_nodes" uses the game status dict to find nodes in the graph adjacent to those of player1, that is, it identifies potential targets to attack. The resulting dict has keys belonging to player and the values are node numbers of potential targets.

In [29]:
def find_adjacent_enemy_nodes(graph_matrix, ownership, player):
    ''' Find nodes adjacent to the player's nodes that are owned by the enemy. '''
    graph = nx.Graph()
    for i, row in enumerate(graph_matrix):
        for j, val in enumerate(row):
            if val == 1:
                graph.add_edge(i, j)

    adjacent_enemies = {}
    for node in ownership:
        if ownership[node] == player:
            enemy_nodes = []
            for neighbor in graph.neighbors(node):
                if ownership[neighbor] != player:
                    enemy_nodes.append(neighbor)
            adjacent_enemies[node] = enemy_nodes
    return adjacent_enemies

{0: [3], 1: [3, 4], 2: [4, 5]}


#### Create subgraphs with enemy nodes
The function "find_subgraphs_with_enemy" uses the adjacent enemy nodes and creates subgraphs consisting of singular enemy nodes and player1 nodes. This constitutes the different types of possible subgraph constellations for attack. With the argument "max_subgraph_size" we can control the allowed sizes for subgraphs in order to evaluate if there is any difference between larger and smaller constellations.

In [30]:
from itertools import combinations

def find_subgraphs_with_enemy(graph_matrix, ownership, player, node_troops, max_subgraph_size=3):
    ''' Find subgraphs formed by Player1's nodes and one enemy node from the adjacent nodes.
        Includes all possible smaller subgraphs (of size >= 2 and <= max_subgraph_size). '''
    graph = nx.Graph()
    for i, row in enumerate(graph_matrix):
        for j, val in enumerate(row):
            if val == 1:
                graph.add_edge(i, j)

    # Only include Player1 nodes with at least 2 troops.
    player_nodes = [node for node, owner in ownership.items()
                    if owner == player and node_troops.get(node, 0) >= 2]

    # Find adjacent enemy nodes.
    adjacent_enemy_nodes = {}
    for player_node in player_nodes:
        enemy_nodes = [neighbor for neighbor in graph.neighbors(player_node)
                       if ownership[neighbor] != player]
        if enemy_nodes:
            adjacent_enemy_nodes[player_node] = enemy_nodes

    # Build subgraphs based on adjacent enemy nodes.
    subgraphs = {}
    for player_node in player_nodes:
        if player_node in adjacent_enemy_nodes:
            for enemy_node in adjacent_enemy_nodes[player_node]:
                full_subgraph_nodes = [enemy_node] + [n for n in player_nodes if graph.has_edge(n, enemy_node)]

                # Generate all possible subsets of the full subgraph.
                for size in range(2, max_subgraph_size + 1):
                    for subset in combinations(full_subgraph_nodes, size):
                        if enemy_node in subset:
                            subset = tuple(sorted(subset))
                            if enemy_node not in subgraphs:
                                subgraphs[enemy_node] = set()
                            subgraphs[enemy_node].add(subset)

    # Convert sets to lists.
    for enemy_node in subgraphs:
        subgraphs[enemy_node] = list(subgraphs[enemy_node])

    return subgraphs

{3: [(0, 1, 3), (0, 3), (1, 3)], 4: [(2, 4), (1, 2, 4), (1, 4)], 5: [(2, 5)]}


#### Generalize subgraph to fetch properties
In order to process the properties of the subgraphs we use "calculate_node_data" to fetch data for a subgraph and "generalize_subgraph" to create a generic representation of the graph so that it matches the form in the dataframe so that its properties can be fetched.

In [31]:
def calculate_node_data(adjacency_matrix, node_ownership, node_troops):
    ''' Calculate the node data dictionary from adjacency matrix, ownership, and troop numbers. '''
    node_data = {}
    for node in range(len(adjacency_matrix)):
        node_data[node] = [2 if node_ownership[node] == 'Player2' else 1, node_troops[node]]
    return node_data

def generalize_subgraph(subgraph, node_data):
    ''' Generalize a subgrap by converting it to generic form as in df. '''
    enemy_node = next(node for node in subgraph if node_data[node][0] == 2)
    player_nodes = [node for node in subgraph if node != enemy_node]

    # Create the generalized subgraph.
    generalized_subgraph = {0: node_data[enemy_node]}
    mapping = {enemy_node: 0}

    for i, player_node in enumerate(sorted(player_nodes, key=lambda n: -node_data[n][1])):
        generalized_subgraph[i + 1] = node_data[player_node]
        mapping[player_node] = i + 1

    return generalized_subgraph, mapping

def get_attacking_nodes(subgraph, ownership, player='Player1'):
    ''' Identify which nodes in the subgraph belong to the specified player. '''
    player_nodes = [node for node in subgraph if ownership.get(node) == player]
    
    return player_nodes

Example node data: 
 {0: [1, 4], 1: [1, 4], 2: [1, 4], 3: [2, 2], 4: [2, 2], 5: [2, 2], 6: [2, 2], 7: [2, 2]}
Example subgraph: 
 (0, 1, 3)
Exampe generalized subgraph 
 ({0: [2, 2], 1: [1, 4], 2: [1, 4]}, {3: 0, 0: 1, 1: 2})


#### Map properties and update game status
The function "analyze_and_map_subgraphs" uses the above functions and maps the properties on to the real subgraph of interest.

In [32]:
import copy
def analyze_and_map_subgraphs(game_status_dict, df, subgraph_size_setting):
    ''' Analyze subgraphs, fetch matching rows from the dataframe, and update the game status dictionary. '''
    
    # Extract data from game_status_dict.
    adjacency_matrix = game_status_dict['original_graph']
    node_ownership = copy.deepcopy(game_status_dict['node_ownership']) 
    node_troops = copy.deepcopy(game_status_dict['node_troops']) 
    aggr_node_path = copy.deepcopy(game_status_dict['aggr_node_path']) 
    aggr_prob = game_status_dict['aggr_prob']
    aggr_attack_type = copy.deepcopy(game_status_dict['aggr_attack_type'])
    aggr_attacking_nodes = copy.deepcopy(game_status_dict['aggr_attacking_nodes'])
    event_probs = copy.deepcopy(game_status_dict['event_probs'])

    # Calculate node data.
    node_data = calculate_node_data(adjacency_matrix, node_ownership, node_troops)

    # Find subgraphs with enemy nodes.
    subgraphs = find_subgraphs_with_enemy(adjacency_matrix, 
                                          node_ownership, 
                                          'Player1', 
                                          game_status_dict['node_troops'],
                                          max_subgraph_size=subgraph_size_setting)

    # Analyze each subgraph.
    subgraph_properties = {}
    for enemy_node, subgraph_list in subgraphs.items():
        subgraph_properties[enemy_node] = []

        for subgraph in subgraph_list:
            # Generalize subgraph and get the mapping.
            generalized_subgraph, mapping = generalize_subgraph(subgraph, node_data)
            initial_state_str = str(generalized_subgraph)
            
            # Get attacking_nodes.
            attacking_nodes = tuple(get_attacking_nodes(subgraph, node_ownership))

            # Fetch all matching rows from the DataFrame.
            matched_rows = df[df['initial_state_str'] == initial_state_str]

            if not matched_rows.empty:
                for _, row in matched_rows.iterrows():
                    filtered_row = {key: row[key] for key in row.keys()
                                    if key in ['successful', 'fraction_of_init']}

                    # Create real_new_state_dict based on new_state_dict and original node numbers.
                    new_state_dict = row['new_state_dict']
                    real_new_state_dict = {original_node: new_state_dict[generic_node]
                                           for original_node, generic_node in mapping.items()
                                           if generic_node in new_state_dict}

                    # Update game_status_dict based on real_new_state_dict.
                    updated_game_status_dict = {
                        'original_graph': game_status_dict['original_graph'],
                        'node_ownership': copy.deepcopy(node_ownership),
                        'node_troops': copy.deepcopy(node_troops),
                        'aggr_node_path': aggr_node_path.copy(),
                        'aggr_attack_type': copy.deepcopy(aggr_attack_type),
                        'aggr_attacking_nodes': copy.deepcopy(aggr_attacking_nodes),
                        'event_probs': copy.deepcopy(event_probs)}

                    # Merge real_new_state_dict into node_ownership and node_troops.
                    for node, state in real_new_state_dict.items():
                        updated_game_status_dict['node_ownership'][node] = 'Player1' if state[0] == 1 else 'Player2'
                        updated_game_status_dict['node_troops'][node] = state[1]

                    # Ensure default values.
                    for node in range(len(updated_game_status_dict['original_graph'])):
                        updated_game_status_dict['node_ownership'].setdefault(node, 'Player2')
                        updated_game_status_dict['node_troops'].setdefault(node, 0)

                    # Update aggr_attack_type.
                    attack_type_used = (len(subgraph) - 1)
                    updated_game_status_dict['aggr_attack_type'].append(attack_type_used)

                    # update aggr_attacking_nodes.
                    updated_game_status_dict['aggr_attacking_nodes'] = updated_game_status_dict['aggr_attacking_nodes'].append(attacking_nodes)

                    # Update aggr_node_path.
                    updated_game_status_dict['aggr_node_path'].append(enemy_node)
                    if aggr_prob == 0:
                        updated_game_status_dict['aggr_prob'] = filtered_row['fraction_of_init']
                    else:
                        updated_game_status_dict['aggr_prob'] = filtered_row['fraction_of_init'] * aggr_prob

                    #Update event_probs.
                    updated_game_status_dict['event_probs'].append(filtered_row['fraction_of_init'])

                    # Add the row's data to the results, tied to the original subgraph.
                    filtered_row.update({
                        'Subgraph': subgraph,
                        'game_status_dict': updated_game_status_dict,
                        'real_subgraph': {node: node_data[node] for node in subgraph},
                        'real_new_state_dict': real_new_state_dict})
                    subgraph_properties[enemy_node].append(filtered_row)
            else:
                # If no match is found, still record the subgraph and generalized state.
                subgraph_properties[enemy_node].append({
                    'Error': 'No match found in DF',
                    'Subgraph': subgraph,
                    'Generalized State': initial_state_str,
                    'game_status_dict': game_status_dict,
                    'real_subgraph': {node: node_data[node] for node in subgraph}})

    return subgraph_properties

#### Aggregate outcomes to create event chains
The function "create_event_chains" could be viewed as the main function of the program using the functions above to create event chains consisting of possible outcomes, aggregating the properties for each new child vertex of the event tree. Key aspects of the event branches are logged and attached to the event chains.

In [33]:
def create_event_chains(game_status_dict, df, max_depth=10, subgraph_size_setting=5):
    '''Generate event chains including both successful and unsuccessful attack scenarios.'''
    import copy

    event_chains = []
    
    # Define milestones for printing.
    milestones = [x * (10 ** 5) for x in range(1, 15)]

    def process_event_chain(current_game_status_dict, depth=0):
        nonlocal milestones

        # Print number of event chains generated.
        while milestones and len(event_chains) >= milestones[0]:
            print(f'{milestones[0]:,} event chains currently generated.')
            milestones.pop(0)

        # Stop recursion if the maximum depth is reached.
        if depth >= max_depth:
            return

        # Ensure default values for the game status.
        current_game_status_dict.setdefault('aggr_prob', 0)
        current_game_status_dict.setdefault('aggr_node_path', [])
        current_game_status_dict.setdefault('aggr_attack_type', [])
        current_game_status_dict.setdefault('aggr_attacking_nodes', [])
        current_game_status_dict.setdefault('event_probs', 0)

        # Check if Player1 conquered all nodes.
        if all(owner == 'Player1' for owner in current_game_status_dict['node_ownership'].values()):
            event_chains.append({
                'depth': depth,
                'termination_reason': 'Player1 conquered all nodes',
                'updated_game_status_dict': current_game_status_dict})
            return

        # Find possible subgraphs where Player1 can attack.
        subgraphs = find_subgraphs_with_enemy(
            current_game_status_dict['original_graph'], 
            current_game_status_dict['node_ownership'], 
            'Player1',
            current_game_status_dict['node_troops'],
            max_subgraph_size=subgraph_size_setting)

        # If no moves are available, terminate.
        if not subgraphs:
            event_chains.append({
                'depth': depth,
                'termination_reason': 'Player1 has no more available moves',
                'updated_game_status_dict': current_game_status_dict})
            return

        # Analyze each possible attack.
        subgraph_results = analyze_and_map_subgraphs(current_game_status_dict, df, subgraph_size_setting)

        move_found = False  # Track if any move (successful or not) was processed.

        for enemy_node, outcomes in subgraph_results.items():
            for outcome in outcomes:

                # Get attacking nodes.
                attacking_nodes = tuple(key for key, value in outcome['real_subgraph'].items() if value[0] == 1)

                event_prob = outcome['fraction_of_init']

                # Process successful attacks.
                if outcome.get('successful') == 1:
                    move_found = True
                    updated_game_status_dict = outcome['game_status_dict']
                    updated_game_status_dict['aggr_node_path'] = current_game_status_dict['aggr_node_path'] + [enemy_node]
                    updated_game_status_dict['aggr_attack_type'] = current_game_status_dict['aggr_attack_type'] + [(len(outcome['Subgraph']) - 1)]
                    updated_game_status_dict['aggr_attacking_nodes'] = current_game_status_dict['aggr_attacking_nodes'] + [attacking_nodes]
                    updated_game_status_dict['event_probs'] = current_game_status_dict['event_probs'] + [event_prob]
                    process_event_chain(copy.deepcopy(updated_game_status_dict), depth + 1)
                    if current_game_status_dict['aggr_prob'] != 0:
                        updated_game_status_dict['aggr_prob'] = current_game_status_dict['aggr_prob'] * event_prob
                    else:
                        updated_game_status_dict['aggr_prob'] = event_prob

                # Process unsuccessful attacks (Player1 attacked but lost).
                elif outcome.get('successful') == 0:
                    move_found = True

                    # Deep copy the current game state.
                    updated_game_status_dict = copy.deepcopy(current_game_status_dict)

                    # Set troops on Player1 nodes in the subgraph to 1.
                    player_nodes_in_subgraph = [node for node in outcome['real_subgraph'] 
                                                if updated_game_status_dict['node_ownership'][node] == 'Player1']
                    for node in player_nodes_in_subgraph:
                        updated_game_status_dict['node_troops'][node] = 1

                    # Update variables.
                    updated_game_status_dict['aggr_node_path'] = current_game_status_dict['aggr_node_path'] + [enemy_node]
                    updated_game_status_dict['aggr_attack_type'] = current_game_status_dict['aggr_attack_type'] + [(len(outcome['Subgraph']) - 1)]
                    updated_game_status_dict['aggr_attacking_nodes'] = current_game_status_dict['aggr_attacking_nodes'] + [attacking_nodes]
                    updated_game_status_dict['event_probs'] = current_game_status_dict['event_probs'] + [event_prob]
                    if current_game_status_dict['aggr_prob'] != 0:
                        updated_game_status_dict['aggr_prob'] = current_game_status_dict['aggr_prob'] * event_prob
                    else:
                        updated_game_status_dict['aggr_prob'] = event_prob

                    # Log the unsuccessful attack.
                    event_chains.append({
                        'depth': depth + 1,
                        'termination_reason': 'Player1 attacked but lost',
                        'updated_game_status_dict': updated_game_status_dict})

                    # Continue recursion with the updated game state.
                    process_event_chain(updated_game_status_dict, depth + 1)

        # If no valid moves were found, terminate.
        if not move_found:
            event_chains.append({
                'depth': depth,
                'termination_reason': 'Player1 has no more available moves',
                'updated_game_status_dict': current_game_status_dict})

    # Start processing the initial game status.
    process_event_chain(game_status_dict)
    return event_chains

#### Process and wrangle data for analysis
We add variables of interest and turn the event chains into a dataframe. 

In [34]:
def calculate_total_troops(game_status_dict):
    ''' Function for calculating total troop numbers. '''
    node_ownership = game_status_dict['node_ownership']
    node_troops = game_status_dict['node_troops']
    total_troops = {'Player1': 0, 'Player2': 0}
    for node, owner in node_ownership.items():
        if owner == 'Player1':
            total_troops['Player1'] += node_troops[node]
        elif owner == 'Player2':
            total_troops['Player2'] += node_troops[node]
    
    return total_troops

initial_troop_numbers = calculate_total_troops(game_status_dict)

def create_simulation_output_df(simulation_output):
    ''' Process simulation output and create dataframe.  '''
    for dict_ in simulation_output:
        dict_['final_troop_numbers'] = calculate_total_troops(dict_['updated_game_status_dict'])
        dict_['final_node_ownership'] = dict_['updated_game_status_dict']['node_ownership']
        dict_['final_node_troop_numbers'] = dict_['updated_game_status_dict']['node_troops']
        dict_['aggr_prob'] = dict_['updated_game_status_dict']['aggr_prob']
        dict_['aggr_node_path'] = dict_['updated_game_status_dict']['aggr_node_path']
        dict_['aggr_attack_type'] = dict_['updated_game_status_dict']['aggr_attack_type']
        dict_['aggr_attacking_nodes'] = dict_['updated_game_status_dict']['aggr_attacking_nodes']
        dict_['event_probs'] = dict_['updated_game_status_dict']['event_probs']

    # Extract useful info from simulation_output and create df.
    filtered_simulation_output = [{key: dict_[key] for key in ['depth', 'aggr_prob', 'aggr_node_path', 'aggr_attack_type', 'aggr_attacking_nodes', 'graph_conquered', 'event_probs',
                                 'final_troop_numbers', 'final_node_ownership', 'final_node_troop_numbers', 'termination_reason'] if key in dict_} for dict_ in simulation_output]
    simulation_output_df = pd.DataFrame(filtered_simulation_output)

    # Add useful variables to df. 
    simulation_output_df['friendly_troop_cost'] = (initial_troop_numbers['Player1'] - 
        simulation_output_df['final_troop_numbers'].apply(lambda x: x['Player1']))
    simulation_output_df['enemy_troop_cost'] = (initial_troop_numbers['Player2'] - 
        simulation_output_df['final_troop_numbers'].apply(lambda x: x['Player2']))
    simulation_output_df['aggr_node_path'] = simulation_output_df['aggr_node_path'].astype(str)
    simulation_output_df['graph_conquered'] = simulation_output_df['final_node_ownership'].apply(
        lambda ownership: all(owner == 'Player1' for owner in ownership.values()))

    return simulation_output_df

### Save data to pickle

In [35]:
def generate_event_chains():
    ''' Function for executing the program. '''
    simulation_output = create_event_chains(game_status_dict, df, max_depth=10)
    simulation_output_df = create_simulation_output_df(simulation_output)
    if not simulation_output_df.empty:
        print('Event chains generated.')
    else:
        print('Could not generate event chains.')
        return
    simulation_output_df.to_pickle('simulation_output_df.pkl')
    print('Event chains has been saved to "simulation_output_df.pkl"')
    
    return simulation_output_df